
# 🏋️ Workout Analytics — Clean Notebook

This notebook loads your workout CSV, cleans the data, and produces **variation-aware** analytics:
- Data loading & cleaning
- Unit tagging (kg vs machine level)
- Weekly trends (kg vs level — separated)
- % changes (kg vs level)
- Volume & intensity trends (by Activity & Variation)
- PRs per Activity & Variation
- Recovery gaps analysis (>7 days)

> Tip: Select the kernel for your Conda env (e.g., `workout-analytics`).


## 0) Configuration

In [ ]:

from pathlib import Path

# Set your input CSV path here:
CSV_PATH = Path("data/Workout Data - Sheet1.csv")  # change if needed

# Output folder for CSVs/plots
OUTDIR = Path("out")
OUTDIR.mkdir(parents=True, exist_ok=True)
print(f"Using CSV: {CSV_PATH.resolve()}")
print(f"Outputs will be saved to: {OUTDIR.resolve()}")


## 1) Imports

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt


## 2) Helper Functions

In [ ]:

def label_row(activity: str, variation: str) -> str:
    v = "(none)" if pd.isna(variation) or str(variation).strip() == "" else str(variation).strip()
    return f"{str(activity).strip()} — {v}"

def weekly_pivot_by_unit(df: pd.DataFrame, unit: str) -> pd.DataFrame:
    """Weekly average by Activity for a given unit ('kg' or 'level')."""
    d = df[df["Weight_Unit"] == unit].copy()
    if d.empty:
        return pd.DataFrame()
    weekly = (
        d.groupby([pd.Grouper(key="Date", freq="W-MON"), "Activity"])["Weight (kg)"]
         .mean()
         .reset_index()
         .sort_values(["Activity", "Date"])
    )
    return weekly.pivot(index="Date", columns="Activity", values="Weight (kg)")

def weekly_pivot_by_unit_with_variation(df: pd.DataFrame, unit: str) -> pd.DataFrame:
    """Weekly average by (Activity, Variation) for a given unit."""
    d = df[df["Weight_Unit"] == unit].copy()
    if d.empty:
        return pd.DataFrame()
    weekly = (
        d.groupby([pd.Grouper(key="Date", freq="W-MON"), "Activity", "Variation"])["Weight (kg)"]
         .mean()
         .reset_index()
    )
    weekly["__label__"] = [label_row(a, v) for a, v in zip(weekly["Activity"], weekly["Variation"])]
    wide = weekly.pivot(index="Date", columns="__label__", values="Weight (kg)")
    return wide.sort_index(axis=1)

def percent_change_from_first_to_last(wide: pd.DataFrame) -> pd.Series:
    """% change from first to last non-NaN week for each column."""
    if wide.empty:
        return pd.Series(dtype=float)
    out = {}
    for col in wide.columns:
        s = wide[col].dropna()
        if len(s) > 1 and s.iloc[0] != 0:
            out[col] = (s.iloc[-1] - s.iloc[0]) / s.iloc[0] * 100.0
    return pd.Series(out).sort_values(ascending=False)

def bar_chart(series: pd.Series, title: str, outfile: Path | None = None):
    if series.empty:
        print(f"No data to plot for: {title}")
        return
    plt.figure(figsize=(10, 5))
    series.round(2).plot(kind="bar")
    plt.title(title)
    plt.ylabel("Percent Change (%)")
    plt.xlabel("Exercise")
    plt.xticks(rotation=45, ha="right")
    plt.grid(True, axis="y")
    plt.tight_layout()
    if outfile:
        plt.savefig(outfile, dpi=160)
    plt.show()


## 3) Load & Clean Data

In [ ]:

# Load
df = pd.read_csv(CSV_PATH)

# Basic cleaning
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
for col in ["Reps", "Weight (kg)", "Set #"]:
    if col in df:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Normalize text
if "Activity" in df:
    df["Activity"] = df["Activity"].astype(str).str.strip().str.lower()
if "Variation" in df:
    df["Variation"] = df["Variation"].astype(str).str.strip().str.lower()
if "Training Type" in df:
    df["Training Type"] = df["Training Type"].astype(str).str.strip().str.lower()

# Example rule: fill bench press missing variation as 'flat'
mask_bp_nan = (df["Activity"] == "bench press") & (df["Variation"].isin([None, '', 'nan']))
df.loc[mask_bp_nan, "Variation"] = "flat"

print("Rows:", len(df))
print("Date range:", df["Date"].min(), "→", df["Date"].max())
df.head()


## 4) Tag Units (kg vs level)

In [ ]:

# Define activities that use numbered machine levels (edit as needed)
machine_number_activities = [
    "leg curls",
    "lever seated row",
    # "seated chest fly",   # moved to kg per your update
    "lat pulldown",
    # "calf raise",         # moved to kg per your update
    "leg extension",
    "rotary trunk twists"
]

df["Weight_Unit"] = df["Activity"].apply(lambda x: "level" if x in machine_number_activities else "kg")

# Manual fixes: ensure specific exercises are kg
df.loc[df["Activity"] == "seated chest fly", "Weight_Unit"] = "kg"
df.loc[df["Activity"] == "chest fly", "Weight_Unit"] = "kg"
df.loc[df["Activity"] == "calf raise", "Weight_Unit"] = "kg"

df.head()


## 5) Overview Stats

In [ ]:

info = {
    "total_sessions": df["Date"].nunique(),
    "unique_activities": df["Activity"].nunique(),
    "unique_variations": df["Variation"].nunique(),
    "total_sets": len(df),
    "avg_reps_per_set": df["Reps"].mean(),
    "avg_logged_weight": df["Weight (kg)"].mean(),
}
pd.Series(info)


## 6) Weekly Progression (KG vs Level — separated)

In [ ]:

weekly_kg = weekly_pivot_by_unit(df, "kg")
weekly_level = weekly_pivot_by_unit(df, "level")

# Plot kg-based trends
plt.figure(figsize=(10,6))
for lift in weekly_kg.columns:
    plt.plot(weekly_kg.index, weekly_kg[lift], marker="o", label=lift)
plt.title("Weekly Progression — KG-based Exercises")
plt.xlabel("Week"); plt.ylabel("Average Weight (kg)")
plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

# Plot level-based trends
plt.figure(figsize=(10,6))
for lift in weekly_level.columns:
    plt.plot(weekly_level.index, weekly_level[lift], marker="o", label=lift)
plt.title("Weekly Progression — Machine Level Exercises")
plt.xlabel("Week"); plt.ylabel("Average Machine Level")
plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()


## 7) % Change (KG vs Level)

In [ ]:

pct_kg = percent_change_from_first_to_last(weekly_kg)
pct_level = percent_change_from_first_to_last(weekly_level)

display(pct_kg.head(10).round(2))
display(pct_level.head(10).round(2))

bar_chart(pct_kg.head(5),  "Top 5 Gains (KG-based)",   OUTDIR / "top5_gains_kg.png")
bar_chart(pct_kg.tail(5),  "Top 5 Drops (KG-based)",   OUTDIR / "top5_drops_kg.png")
bar_chart(pct_level.head(5), "Top 5 Gains (Machine Level)", OUTDIR / "top5_gains_level.png")
bar_chart(pct_level.tail(5), "Top 5 Drops (Machine Level)", OUTDIR / "top5_drops_level.png")


## 8) Volume & Intensity (by Activity & Variation; KG only)

In [ ]:

df_kg = df[df["Weight_Unit"] == "kg"].copy()
df_kg["Volume"] = df_kg["Reps"] * df_kg["Weight (kg)"]

# Weekly Volume
weekly_volume = (
    df_kg.groupby([pd.Grouper(key="Date", freq="W-MON"), "Activity", "Variation"])["Volume"]
        .sum().reset_index()
)
weekly_volume["Label"] = [label_row(a, v) for a, v in zip(weekly_volume["Activity"], weekly_volume["Variation"])]

plt.figure(figsize=(10,6))
for lbl in weekly_volume["Label"].unique():
    d = weekly_volume[weekly_volume["Label"] == lbl]
    plt.plot(d["Date"], d["Volume"], marker="o", label=lbl)
plt.title("Weekly Training Volume — Activity & Variation")
plt.ylabel("Total Volume (kg × reps)"); plt.xlabel("Week")
plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

# Weekly Intensity (Avg per set)
weekly_intensity = (
    df_kg.groupby([pd.Grouper(key="Date", freq="W-MON"), "Activity", "Variation"])["Weight (kg)"]
        .mean().reset_index()
)
weekly_intensity["Label"] = [label_row(a, v) for a, v in zip(weekly_intensity["Activity"], weekly_intensity["Variation"])]

plt.figure(figsize=(10,6))
for lbl in weekly_intensity["Label"].unique():
    d = weekly_intensity[weekly_intensity["Label"] == lbl]
    plt.plot(d["Date"], d["Weight (kg)"], marker="o", label=lbl)
plt.title("Weekly Intensity (Avg Weight per Set) — Activity & Variation")
plt.ylabel("Weight (kg)"); plt.xlabel("Week")
plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()


## 9) Personal Records (by Activity & Variation; KG only)

In [ ]:

pr_table = (
    df_kg.groupby(["Activity", "Variation"])["Weight (kg)"]
        .max().sort_values(ascending=False)
)
print("🏆 Personal Records (kg) by Activity & Variation:")
display(pr_table.to_frame("PR (kg)"))

# Bar plot (optional; can be large if many exercises)
pr_plot = pr_table.copy()
pr_plot.index = [label_row(a, v) for a, v in pr_plot.index]
plt.figure(figsize=(10,6))
pr_plot.plot(kind="bar")
plt.title("Personal Records (kg) — Activity & Variation")
plt.ylabel("Max Weight (kg)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Export
pr_table.to_csv(OUTDIR / "prs_kg_by_activity_variation.csv")


## 10) Recovery Effect (>7 day gaps; KG only)

In [ ]:

df_sorted = df_kg.sort_values("Date").copy()
df_sorted["Gap_days"] = df_sorted["Date"].diff().dt.days
gaps = df_sorted[df_sorted["Gap_days"] > 7]["Date"]

if not gaps.empty:
    print("\n🛌 Recovery Effect Analysis (overall kg across last/next 5 sets):")
    for gap_date in gaps:
        before_vals = df_sorted[df_sorted["Date"] < gap_date].tail(5)["Weight (kg)"]
        after_vals  = df_sorted[df_sorted["Date"] > gap_date].head(5)["Weight (kg)"]
        if len(before_vals) == 0 or before_vals.mean() == 0 or len(after_vals) == 0:
            continue
        change = (after_vals.mean() - before_vals.mean()) / before_vals.mean() * 100
        print(f"Gap starting {gap_date.date()}: Avg kg change = {change:.2f}% "
              f"(before={before_vals.mean():.1f}, after={after_vals.mean():.1f})")
else:
    print("\nNo gaps over 7 days found — consistent training!")


## 11) Save Cleaned CSV (optional)

In [ ]:

# Write a cleaned copy for future use
clean_path = OUTDIR / "Workout Data - cleaned.csv"
df.to_csv(clean_path, index=False)
print(f"Saved cleaned CSV to: {clean_path.resolve()}")
